# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.58,63,3,2.49,SH,1729042721
1,1,port-aux-francais,-49.3500,70.2167,0.92,66,74,16.03,TF,1729042723
2,2,mayya,61.7333,130.2811,-2.22,74,50,1.44,RU,1729042724
3,3,saipan,15.1355,145.7010,30.39,79,40,0.00,MP,1729042725
4,4,floro,61.5996,5.0328,9.98,71,100,9.26,NO,1729042726


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM',  # OpenStreetMap tiles
    size='Humidity', 
    color='City', 
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity'],
    cmap='Category20', 
    frame_width=800, 
    frame_height=600, 
    title="City Humidity Map")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) & 
    (city_data_df['Max Temp'] <= 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,san ignacio,17.1561,-89.0714,26.08,89,0,0.00,BZ,1729042827
81,81,tura,25.5198,90.2201,23.62,87,0,0.96,IN,1729042832
117,117,iquitos,-3.7481,-73.2472,26.02,78,0,1.54,PE,1729042672
179,179,san patricio,28.0170,-97.5169,24.97,61,0,3.66,US,1729042947
205,205,tahta,26.7693,31.5021,24.05,43,0,3.42,EG,1729042978


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
77,san ignacio,BZ,17.1561,-89.0714,89,
81,tura,IN,25.5198,90.2201,87,
117,iquitos,PE,-3.7481,-73.2472,78,
179,san patricio,US,28.0170,-97.5169,61,
205,tahta,EG,26.7693,31.5021,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters radius to search for hotels
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # We only want the closest hotel
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san ignacio - nearest hotel: Rainforest Haven Inn
tura - nearest hotel: Hotel Polo Orchid
iquitos - nearest hotel: Marfil del Amazonas
san patricio - nearest hotel: No hotel found
tahta - nearest hotel: No hotel found
galveston - nearest hotel: Comfort Inn & Suites Texas City
arar - nearest hotel: دار أوتيل Dar Otel
sarpol-e zahab - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
la cruz - nearest hotel: Hotel Las Palmas
deir ez-zor - nearest hotel: فندق السعيد
laguna - nearest hotel: Holiday Inn Express & Suites
teguise - nearest hotel: Finca Las Laderas
al jawf - nearest hotel: No hotel found
gasa - nearest hotel: مسجد الفتح المبين
tindouf - nearest hotel: محمد بوسبي
al qusiyah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
77,san ignacio,BZ,17.1561,-89.0714,89,Rainforest Haven Inn
81,tura,IN,25.5198,90.2201,87,Hotel Polo Orchid
117,iquitos,PE,-3.7481,-73.2472,78,Marfil del Amazonas
179,san patricio,US,28.0170,-97.5169,61,No hotel found
205,tahta,EG,26.7693,31.5021,43,No hotel found
224,galveston,US,29.3669,-94.9669,80,Comfort Inn & Suites Texas City
225,arar,SA,30.9753,41.0381,24,دار أوتيل Dar Otel
276,sarpol-e zahab,IR,34.4611,45.8626,28,No hotel found
356,crane,US,31.3974,-102.3501,20,No hotel found
360,uturoa,PF,-16.7333,-151.4333,63,Hawaiki Nui hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM',  # OpenStreetMap tiles
    size='Humidity', 
    color='City', 
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity'],
    cmap='Category20', 
    frame_width=800, 
    frame_height=600, 
    title="Hotels Found for Ideal Cities")

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)